# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [3]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [4]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [5]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [6]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - dc7842bb


In [7]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [8]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [9]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [10]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_2", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [11]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: c:\Users\mjelic\ai_lab\code\labs\07_Deep_Agents\workspace


In [12]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] personalized_sleep_improvement_plan.md (4907 bytes)
[DIR] research
[FILE] stress_management_guide.md (12955 bytes)


In [13]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (252 bytes)


In [14]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [15]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: c:\Users\mjelic\ai_lab\code\labs\07_Deep_Agents\workspace


In [16]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Personalized Sleep Improvement Plan is Complete! 

I've created a comprehensive, evidence-based sleep improvement plan specifically tailored to your challenges and saved it to `/my_sleep_improvement_plan.md`. 

### Key Highlights of Your Plan:

**🎯 Addresses Your Specific Issues:**
- **Inconsistent bedtime (10pm-1am)**: 4-phase approach to gradually consolidate your sleep schedule
- **Phone use in bed**: Physical barriers and digital sunset protocol
- **Morning fatigue**: Morning light therapy and circadian rhythm optimization

**📅 4-Phase Timeline (12 weeks):**
1. **Week 1**: Foundation reset - remove phone, fix wake time (7:00am daily)
2. **Weeks 2-3**: Digital sunset and bedtime consolidation
3. **Weeks 4-6**: Advanced sleep optimization techniques
4. **Weeks 7-12**: Long-term habit mastery

**🚀 Start Tonight Actions:**
- Set up phone charging station outside bedroom
- Choose 7:00am as your consistent wake time
- Set bedroom temperature to 68°F or below

**📊 

In [17]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep improvement strategies (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save the plan to a file (completed)
✅ [todo_9] Provide summary and next steps (completed)
✅ [todo_6] Research inconsistent bedtime solutions (completed)
✅ [todo_8] Research phone use in bed interventions (completed)
✅ [todo_10] Research morning fatigue solutions (completed)
✅ [todo_12] Research circadian rhythm regulation (completed)
✅ [todo_14] Research environmental optimization (completed)
✅ [todo_16] Research morning routine optimization (completed)
✅ [todo_18] Compile actionable implementation plan (completed)


Workspace contents:
  [FILE] comprehensive_sleep_improvement_guide.md (12563 bytes)
  [FILE] my_sleep_improvement_plan.md (6396 bytes)
  [FILE] personalized_sleep_improvement_plan.md (4907 bytes)
  [DIR] research/
  [FILE] stress_management_guide.md 

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
Use a todo-planning architecture when:
- the task has multiple steps
- it involves tools
- it requires state persistence
- it needs robustness
- there are potential failure points

Planning overhead becomes problematic when:
- the tasks are simple
- the tasks are short, single-step tool uses
- the environment is slow or rate-limited (for example, I once received a 529 "overloaded" error)


You need to strike a balance between overly granular and overly coarse structure.
Todo items should be:
- semantically coherent
- self-contained (able to be executed independently)


If the agent creates todos but never completes them, this can lead to:
- inconsistent state
- infinite loops
- re-planning without ever reaching completion


## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
In the Prompt: 

- allergies, medications, conditions

- current goal and short active plan

- summary of metrics 



In Files / Storage

- large health document (chunked and indexed for retrieval)

- full metrics history  

- user profile



Should Never Be Offloaded

- safety constraints (allergies, contraindications, conditions)



---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [18]:
### YOUR CODE HERE ###

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

workspace_path = WORKSPACE.absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True
)




In [19]:
# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")

@tool
def read_health_wellness_guide() -> str:
    """Read the HealthWellnessGuide.txt from the data folder (read-only)."""
    guide_path = Path("data/HealthWellnessGuide.txt")
    if not guide_path.exists():
        return f"File not found: {guide_path}"
    return guide_path.read_text(encoding="utf-8", errors="ignore")



In [ ]:
# Step 3: Create the agent with a research-focused system prompt

custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_health_wellness_guide,  # tool za čitanje guide-a
]

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  
    system_prompt="""
You are a Wellness Research Agent.

Your job: research the user's wellness topic and produce a structured markdown report.

Workflow (REQUIRED):
1) Create a todo list for the research process using write_todos.
   - Include: read guide, extract key points, outline report, draft strategies, finalize + save report.
2) Use read_health_wellness_guide to read the HealthWellnessGuide.txt (data folder).
3) Work through todos in order. Before working on a todo, mark it in_progress, and when done mark completed.
4) Save a structured markdown report to a file in the workspace using the built-in write_file tool.
   - Use a clear structure with headings, bullet points, and a short summary.
5) At the end, show the user where the report was saved and provide a brief summary.

Output requirements for the report:
- Title
- Short executive summary
- At least 5 evidence-based strategies (each with: what it is, why it works, how to apply, cautions)
- Optional: quick checklist, 7-day starter plan
"""
)

print("Research agent created!")
print("Workspace sandbox:", workspace_path)




Research agent created!
Workspace sandbox: c:\Users\mjelic\ai_lab\code\labs\07_Deep_Agents\workspace


In [ ]:
# Step 4: Test with the stress management research task

TODO_STORE.clear()

result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."
    }]
})

print("\nAgent response:\n")
print(result["messages"][-1].content)

print("\nTodos after run:\n")
print(list_todos.invoke({}))

print("\nWorkspace contents:\n")

for p in sorted(WORKSPACE.rglob("*")):
    if p.is_file():
        print(f"[FILE] {p.relative_to(WORKSPACE)} ({p.stat().st_size} bytes)")


Agent response:

Perfect! I've successfully completed your comprehensive stress management guide. Here's what I've created for you:

## 📋 Research Project Complete!

Your comprehensive stress management guide has been saved to: **`/comprehensive_stress_management_guide.md`**

## 📊 What's Included:

### **7 Evidence-Based Strategies:**
1. **Box Breathing** - Immediate nervous system regulation
2. **Progressive Muscle Relaxation** - Physical tension release
3. **Mindfulness Meditation** - Long-term brain changes and resilience
4. **Regular Physical Exercise** - Natural endorphin boost and stress hormone reduction
5. **Grounding Technique (5-4-3-2-1)** - Present-moment anchoring for anxiety
6. **Sleep Hygiene Optimization** - Breaking the stress-sleep cycle
7. **Social Connection & Boundary Setting** - Relationship-based stress management

### **Practical Tools:**
- ✅ Emergency 5-minute stress relief checklist
- ✅ Daily stress prevention habit tracker
- ✅ Complete 7-day starter implement

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [22]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [23]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [24]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I've delivered:

## ✅ **Complete Project Summary:**

### **1. Scientific Research Integration**
- **Circadian rhythm optimization** with specific light exposure recommendations (10,000 lux within 30 minutes)
- **Exercise science** showing neurochemical benefits (endorphins, serotonin, BDNF)
- **Nutritional research** on morning metabolism and blood sugar stability
- **Psychological studies** on mindfulness and stress reduction
- All backed by credible sources and specific mechanisms

### **2. Comprehensive Three-Pillar Framework**
- **🏃‍♂️ Movement & Exercise:** High/moderate/low-impact options with sample routines
- **🥗 Nutrition & Hydration:** Strategic timing, macronutrient balance, 5 breakfast recipes
- **🧠 Mindset & Mental Preparation:** Meditation, journaling, goal-setting, digital boundaries

### **3. Six Ready-to-Use Routine Templates**
- **The Quick 

In [25]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Save the guide as a markdown file (completed)

Generated files in workspace:
  [FILE] comprehensive_sleep_improvement_guide.md (12563 bytes)
  [FILE] comprehensive_stress_management_guide.md (11126 bytes)
  [FILE] morning_energy_guide.md (37297 bytes)
  [FILE] morning_routine_guide.md (16934 bytes)
  [FILE] my_sleep_improvement_plan.md (6396 bytes)
  [FILE] personalized_sleep_improvement_plan.md (4907 bytes)
  [DIR] research/
  [FILE] stress_management_guide.md (12955 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [26]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [27]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [28]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [29]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! Great to see you again. I can see from your profile that your primary goal is to improve energy levels and your secondary goal is better sleep. I also note that you prefer morning workouts, have mild anxiety, and like detailed communication.

Based on your specific goals and conditions, here's a personalized exercise routine I recommend for you:

## **Morning Energy-Boosting Routine (30-45 minutes)**

### **Phase 1: Gentle Wake-Up (5-10 minutes)**
- **Light stretching or yoga flow**: Start with gentle movements to wake up your body
- **Deep breathing exercises**: 2-3 minutes of box breathing (4-4-4-4 count) to help manage anxiety and center yourself

### **Phase 2: Cardiovascular Energy Boost (15-20 minutes)**
- **Moderate-intensity cardio**: Choose one you enjoy:
  - Brisk walking or light jogging
  - Dancing to upbeat music
  - Cycling (stationary or outdoor)
  - Swimming (if available)
- Keep intensity at 60-70% of your maximum heart rate - you should be abl

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [30]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [31]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [32]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [33]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [34]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Now let me provide you with your personalized action plan based on this assessment:

## Your Wellness Assessment Results

### **Current Wellness State:**
You're in a common but manageable situation for office workers. Your sedentary lifestyle is the primary driver affecting both your weight and sleep quality. The good news is that you have a strong foundation with no major health conditions and a vegetarian diet that can support your goals.

### **Key Finding:** 
Your three concerns (weight, sleep, sedentary lifestyle) are interconnected - improving one will help the others!

---

## **Immediate Action Items (Start Today):**

1. **Set hourly movement alarms** - Stand and walk for 2-3 minutes every hour during work
2. **Create a wind-down routine** - No screens 1 hour before bed, dim lights, try gentle stretching
3. **Track your baseline** - Record current sleep hours and daily steps (phone or simple app)

## **Short-Term Goals (1-2 weeks):**

1. **Establish morning move

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [35]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents-cli 0.0.24
deepagents (SDK) 0.4.1


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [36]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [37]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [38]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
## Your Complete 2-Week Wellness Program is Ready! 🎉

Alex, I've created a comprehensive wellness program tailored specifically for you, considering your vegetarian diet, mild anxiety, preference for morning exercise, and busy schedule. Here's what I've prepared:

### 📁 **Your Reference Files:**
1. **`alex_complete_wellness_program.md`** - Your integrated master plan with daily schedules
2. **`alex_exercise_plan.md`** - Detailed workout routines (3x/week, 30 mins each)
3. **`alex_nutrition_plan.md`** - Complete vegetarian meal plans with recipes and shopping lists
4. **`alex_mindfulness_plan.md`** - Stress management and sleep optimization techniques

### 🎯 **Program Highlights:**

**Exercise Component:**
- 6 progressive workouts designed for consistency and energy building
- Combines strength training, cardio, and flexibility
- Specifically designed for anxiety-friendly, confidence-building exercises

**Nutrition Component:**
- 14 days of complete vegetarian m

In [39]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create comprehensive exercise plan (3x/week, 30min) (completed)
✅ [todo_3] Design vegetarian meal plan for 2 weeks (completed)
✅ [todo_5] Develop stress management and sleep optimization plan (completed)
✅ [todo_7] Create integrated 2-week wellness program document (completed)
✅ [todo_9] Save individual specialist plans as separate reference files (completed)

GENERATED FILES
  [FILE] alex_complete_wellness_program.md (10017 bytes)
  [FILE] alex_exercise_plan.md (5010 bytes)
  [FILE] alex_mindfulness_plan.md (10737 bytes)
  [FILE] alex_nutrition_plan.md (8379 bytes)
  [FILE] comprehensive_sleep_improvement_guide.md (12563 bytes)
  [FILE] comprehensive_stress_management_guide.md (11126 bytes)
  [FILE] morning_energy_guide.md (37297 bytes)
  [FILE] morning_routine_guide.md (16934 bytes)
  [FILE] my_sleep_improvement_plan.md (6396 bytes)
  [FILE] personalized_sleep_improvement_plan.md (4907 bytes)
  [DIR] research/
  [FILE] stress_management_guide.md (12955 by

In [42]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))

if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    
    text = files[0].read_text(encoding="utf-8", errors="replace")
    
    if len(text) > 2000:
        print(text[:2000] + "...")
    else:
        print(text)


Contents of alex_complete_wellness_program.md:
# Alex's Complete 2-Week Wellness Program
*Integrated Exercise, Nutrition, and Mindfulness Plan*

## Program Overview
This comprehensive wellness program integrates your exercise routine, vegetarian nutrition plan, and stress/sleep management into a cohesive 2-week transformation. Each component works synergistically to improve your energy levels, manage work stress, and enhance sleep quality.

## Your Wellness Goals
- ✅ Build consistent exercise routine (3x/week, 30 mins)
- ✅ Improve energy through optimized vegetarian nutrition
- ✅ Manage work stress and improve sleep quality
- ✅ Address mild anxiety through mindful practices

---

## Week 1: Foundation Building

### Monday - Week 1
**Morning Routine (30 mins total):**
- 5:30 AM: Wake up + 5-minute mindful breathing
- 6:00 AM: Pre-workout snack (half banana with almond butter)
- 6:30 AM: **Exercise Session 1** - Full Body Strength & Cardio
- 7:00 AM: Post-workout breakfast (overnight oa

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
Share tools when:
- we want consistent behavior and clear rules (for example, for writing to files)
- the tools are low-risk (such as search, retrieval, etc.)


Use distinct tools when:
- a specific safety rule is involved
- the tools are high-risk
- the tools provide specialized capabilities.

We should use the strongest model for planning and safety checks, and cheaper models for drafting.

Regarding granularity:

The answer is similar to the first question in this chapter about granularity.
It is necessary to find the right balance. Specialization should be meaningful and stable. It should not be too coarse, but also not too fine, as too much message passing and context handoff can create unnecessary overhead.

A good example of subagent roles:
Planner
Researcher
Writer
Safety / Verifier

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:

Safety Guardrails for Health Advice:
- medical safety layer – contraindication checks using the user profile (allergies, medications, conditions, pregnancy, chronic diseases)

 - the agent must switch to emergency guidance and recommend professional help in case of urgent situations (for example, a severe allergic reaction)


Replace the in-memory TODO_STORE with a proper database (tables for user data, metrics, etc.).


Multi-User Isolation and Data Protection:
- every read or write operation must be strictly scoped to a specific user
- no shared paths within a per-user workspace for user files
- access control for user data
- respect for user privacy, including encryption and anonymization

Monitoring and Observability:
- logging agent access to user data
- metrics such as latency, tool-call counts, and token usage


Cost Management with Subagents :
- as mentioned previously, use the strongest model for planning and safety checks, and cheaper models for drafting
 -token budgeting per request and per user, with stop conditions when limits are exceeded
- caching
- enforcing appropriate todo granularity




---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [44]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from pathlib import Path
from typing import List, Dict, Any, Literal
import json
from datetime import date, datetime, timedelta

WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

workspace_path = WORKSPACE.absolute()
filesystem_backend = FilesystemBackend(root_dir=str(workspace_path), virtual_mode=True)

MODEL_STRONG = "anthropic:claude-sonnet-4-20250514"  # planning/safety
MODEL_CHEAP = "anthropic:claude-3-5-haiku-latest"    # drafting/summary (ako nije dostupno u labu, promijeni u sonnet)

STATE_DIR = WORKSPACE / "state"
STATE_DIR.mkdir(exist_ok=True)

PROFILE_PATH = STATE_DIR / "user_profile.json"
TODOS_PATH = STATE_DIR / "todos.json"

CHECKINS_DIR = WORKSPACE / "checkins"
CHECKINS_DIR.mkdir(exist_ok=True)

REPORTS_DIR = WORKSPACE / "reports"
REPORTS_DIR.mkdir(exist_ok=True)

plan_subagent = create_deep_agent(
    model=init_chat_model(MODEL_STRONG),
    tools=[],
    backend=filesystem_backend,
    system_prompt="""
You are a 30-Day Wellness Challenge Planner.

Create a personalized 30-day plan based on the user's preferences and constraints.
Return:
- a short profile summary (goals, constraints)
- a 30-day plan grouped by weeks
- each day: 2-4 actions (simple, realistic)
Keep it practical and safe (general wellness, not medical advice).
"""
)

summary_subagent = create_deep_agent(
    model=init_chat_model(MODEL_CHEAP),
    tools=[],
    backend=filesystem_backend,
    system_prompt="""
You are a Weekly Wellness Summary Writer.

Given a set of daily check-ins for a week, produce a structured markdown weekly report:
- Overview
- Wins
- Challenges
- Trends
- Next week adjustments
Be concise and actionable.
"""
)



In [ ]:

# Step 2: Create any additional tools you need
def _read_json(path: Path, default: Any) -> Any:
    if not path.exists():
        return default
    try:
        return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        return default

def _write_json(path: Path, obj: Any) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")

@tool
def load_user_profile() -> str:
    """Load the persistent user profile (preferences/memory)."""
    profile = _read_json(PROFILE_PATH, default={
        "name": None,
        "goals": [],
        "preferences": {
            "workout_style": None,  
            "diet_style": None,      
            "time_available_min": None,
            "sleep_goal": None
        },
        "constraints": [],          
        "last_updated": None
    })
    return json.dumps(profile, ensure_ascii=False, indent=2)

@tool
def save_user_profile(profile_json: str) -> str:
    """Save/overwrite the persistent user profile (preferences/memory)."""
    try:
        profile = json.loads(profile_json)
    except json.JSONDecodeError:
        return "Invalid JSON. Could not save profile."
    profile["last_updated"] = datetime.now().isoformat()
    _write_json(PROFILE_PATH, profile)
    return f"Saved profile to state/user_profile.json"

@tool
def create_todos(todos: List[dict]) -> str:
    """Create persistent todos (planning)."""
    store = _read_json(TODOS_PATH, default={"todos": []})
    next_id = 1 + max([t["id"] for t in store["todos"]], default=0)

    created = []
    for item in todos:
        todo = {
            "id": next_id,
            "title": item.get("title", "Untitled"),
            "description": item.get("description", ""),
            "status": "pending"
        }
        store["todos"].append(todo)
        created.append(next_id)
        next_id += 1

    _write_json(TODOS_PATH, store)
    return f"Created {len(created)} todos: {created}"

@tool
def update_todo_status(todo_id: int, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update a persistent todo's status."""
    store = _read_json(TODOS_PATH, default={"todos": []})
    for t in store["todos"]:
        if t["id"] == todo_id:
            t["status"] = status
            _write_json(TODOS_PATH, store)
            return f"Updated todo {todo_id} -> {status}"
    return f"Todo {todo_id} not found"

@tool
def list_todos() -> str:
    """List all persistent todos."""
    store = _read_json(TODOS_PATH, default={"todos": []})
    if not store["todos"]:
        return "No todos."
    lines = []
    emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
    for t in store["todos"]:
        lines.append(f"{emoji.get(t['status'],'❓')} [{t['id']}] {t['title']} ({t['status']})")
    return "\n".join(lines)

@tool
def save_daily_checkin(day_iso: str, note_md: str) -> str:
    """Store daily check-in notes (context management)."""
    try:
        _ = date.fromisoformat(day_iso)
    except ValueError:
        return "Invalid date. Use YYYY-MM-DD."
    path = CHECKINS_DIR / f"{day_iso}.md"
    path.write_text(note_md, encoding="utf-8")
    return f"Saved check-in to checkins/{day_iso}.md"

@tool
def load_checkins_range(start_iso: str, end_iso: str) -> str:
    """Load check-ins in [start, end] date range."""
    try:
        start = date.fromisoformat(start_iso)
        end = date.fromisoformat(end_iso)
    except ValueError:
        return "Invalid date range. Use YYYY-MM-DD."
    if end < start:
        return "End date must be >= start date."

    items = []
    cur = start
    while cur <= end:
        p = CHECKINS_DIR / f"{cur.isoformat()}.md"
        if p.exists():
            items.append({"date": cur.isoformat(), "content": p.read_text(encoding="utf-8", errors="replace")})
        cur += timedelta(days=1)

    return json.dumps(items, ensure_ascii=False, indent=2)

@tool
def call_plan_subagent(user_request: str) -> str:
    """Call the planner subagent to generate the 30-day plan text."""
    res = plan_subagent.invoke({"messages": [{"role": "user", "content": user_request}]})
    return res["messages"][-1].content

@tool
def call_summary_subagent(week_checkins_json: str) -> str:
    """Call the summary subagent to write a weekly report."""
    res = summary_subagent.invoke({"messages": [{"role": "user", "content": week_checkins_json}]})
    return res["messages"][-1].content



In [ ]:
# Step 3: Build the main coordinator agent
tools = [
    load_user_profile,
    save_user_profile,
    create_todos,
    update_todo_status,
    list_todos,
    save_daily_checkin,
    load_checkins_range,
    call_plan_subagent,
    call_summary_subagent,
]

wellness_coach = create_deep_agent(
    model=init_chat_model(MODEL_STRONG),
    tools=tools,
    backend=filesystem_backend,  # enables built-in file tools write_file/read_file/etc in workspace
    system_prompt="""
You are a Production-Style Wellness Coach (Deep Agent) that runs a 30-Day Wellness Challenge.

You MUST use all four elements:
1) Planning: create todos for the full 30-day challenge creation workflow and update statuses as you complete them.
2) Context management: store daily check-in notes to files (use save_daily_checkin) and use them to adapt recommendations.
3) Subagents: use call_plan_subagent for plan generation and call_summary_subagent for weekly summaries.
4) Memory: load and update user preferences using load_user_profile / save_user_profile so preferences persist across interactions.

Output rules:
- Keep recommendations realistic and safe (general wellness).
- If user preferences are missing, make reasonable defaults AND store them in the profile.
- Save the 30-day plan as markdown in reports/30_day_plan.md using the built-in write_file tool.
- Save weekly summary reports as reports/weekly_summary_YYYY-MM-DD_to_YYYY-MM-DD.md using write_file.
- Always update todo statuses: pending -> in_progress -> completed.
"""
)

print("Wellness Coach created. Workspace:", workspace_path)




✅ Wellness Coach created. Workspace: c:\Users\mjelic\ai_lab\code\labs\07_Deep_Agents\workspace


In [48]:
# Step 4: Test with a user creating their 30-day challenge
create_prompt = """
Create my 30-Day Wellness Challenge.
Preferences:
- I can spend 20 minutes per day
- I prefer walking + simple strength exercises at home
- Goal: reduce stress and sleep better
Constraints:
- I don't want caffeine after 2pm
Please generate the plan and save it.
"""

result = wellness_coach.invoke({"messages": [{"role": "user", "content": create_prompt}]})
print(result["messages"][-1].content)

print("\n--- TODOS ---")
print(list_todos.invoke({}))



## 🎉 Your 30-Day Wellness Challenge is Ready!

I've successfully created your personalized **30-Day Stress Relief & Sleep Improvement Challenge**! Here's what I've accomplished:

### ✅ **Challenge Features Perfectly Matched to Your Needs:**

**🎯 Your Goals Addressed:**
- **Reduce stress**: Daily breathing exercises, mindfulness walks, progressive muscle relaxation
- **Sleep better**: Sleep hygiene setup, bedtime routines, no-screen policies, 2PM caffeine cutoff

**⏰ Time Commitment Respected:**
- Every day fits within your 20-minute maximum
- Options for 15-minute days when you're super busy
- No time wasted - efficient, focused activities

**🏠 Perfect for Your Preferences:**
- **Walking**: Indoor and outdoor options for every weather condition
- **Simple strength**: All bodyweight exercises, no equipment needed
- **At-home focus**: Everything can be done in your living space

**📊 Progressive Structure:**
- **Week 1**: Foundation building (gentle intro)
- **Week 2**: Adding strength an

In [49]:
# Step 5: Simulate a daily check-in and adaptation
checkin_prompt = """
Daily check-in for 2026-02-21:
- I did the walk, but I felt anxious in the evening.
- Sleep: 6h, woke up twice.
Please log this check-in and adjust tomorrow's recommendation slightly.
Also, if useful, update my preferences for next time.
"""

result2 = wellness_coach.invoke({"messages": [{"role": "user", "content": checkin_prompt}]})
print("\n--- CHECK-IN RESPONSE ---")
print(result2["messages"][-1].content)

print("\n--- Workspace reports/checkins files ---")
for p in sorted(WORKSPACE.rglob("*")):
    if p.is_file():
        print("[FILE]", p.relative_to(WORKSPACE))



--- CHECK-IN RESPONSE ---
## 📊 What I've Updated:

1. **Your Profile:** Added notes about evening anxiety management needs and sleep disruption patterns
2. **Tomorrow's Focus:** Emphasis on evening wind-down routine and anxiety management techniques
3. **Continuity:** Keeping what works (your walks, caffeine timing) while addressing what needs improvement

How does this adjusted plan feel for tomorrow? I'm here to help you check in again and continue refining your wellness approach based on what you experience.

--- Workspace reports/checkins files ---
[FILE] 30_day_wellness_challenge.md
[FILE] alex_complete_wellness_program.md
[FILE] alex_exercise_plan.md
[FILE] alex_mindfulness_plan.md
[FILE] alex_nutrition_plan.md
[FILE] checkins\2026-02-21.md
[FILE] comprehensive_sleep_improvement_guide.md
[FILE] comprehensive_stress_management_guide.md
[FILE] morning_energy_guide.md
[FILE] morning_routine_guide.md
[FILE] my_sleep_improvement_plan.md
[FILE] personalized_sleep_improvement_plan.md
[

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)